In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt

2023-03-04 12:47:52.408840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 12:47:53.070115: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-04 12:47:53.070172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-04 12:47:53.070178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
IMG_SIZE = 128;
NUM_CLASSES = 10;
batch_size = 16;
size = (IMG_SIZE,IMG_SIZE);

In [6]:
def resize(image, label):
    resized = tf.image.resize(image, size)
    luminance = tf.image.rgb_to_grayscale(resized)
    four_channel = tf.concat((luminance,resized),2)
    return four_channel, label

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [7]:
(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

In [8]:
ds_train = ds_train.map(resize)
ds_test = ds_test.map(resize)

ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

# ds_train = ds_train.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))
# ds_test = ds_test.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))


In [12]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 4))
x = img_augmentation(inputs)
outputs = MobileNetV2(
    include_top=True,
    weights=None,
    alpha=0.5,
    classes=10,
    input_shape=(128,128,4),
    
)(x)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 4)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 128, 128, 4)      0         
 l)                                                              
                                                                 
 mobilenetv2_0.50_128 (Funct  (None, 10)               719178    
 ional)                                                          
                                                                 
Total params: 719,178
Trainable params: 700,634
Non-trainable params: 18,544
_________________________________________________________________


In [14]:
epochs = 5  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5


591/591 - 103s - loss: 2.1608 - accuracy: 0.2446 - val_loss: 2.3310 - val_accuracy: 0.0987 - 103s/epoch - 174ms/step
Epoch 2/5
591/591 - 74s - loss: 1.9190 - accuracy: 0.3268 - val_loss: 2.5471 - val_accuracy: 0.0987 - 74s/epoch - 126ms/step
Epoch 3/5
591/591 - 74s - loss: 1.7937 - accuracy: 0.3764 - val_loss: 2.7473 - val_accuracy: 0.0987 - 74s/epoch - 126ms/step
Epoch 4/5
591/591 - 75s - loss: 1.6901 - accuracy: 0.4196 - val_loss: 2.7619 - val_accuracy: 0.0987 - 75s/epoch - 127ms/step
Epoch 5/5
591/591 - 74s - loss: 1.6124 - accuracy: 0.4471 - val_loss: 2.8758 - val_accuracy: 0.0987 - 74s/epoch - 126ms/step


In [15]:
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5
591/591 - 74s - loss: 1.5325 - accuracy: 0.4747 - val_loss: 3.1630 - val_accuracy: 0.1064 - 74s/epoch - 125ms/step
Epoch 2/5
591/591 - 74s - loss: 1.4917 - accuracy: 0.4913 - val_loss: 3.0898 - val_accuracy: 0.1673 - 74s/epoch - 126ms/step
Epoch 3/5
591/591 - 75s - loss: 1.4347 - accuracy: 0.5120 - val_loss: 2.1478 - val_accuracy: 0.3258 - 75s/epoch - 126ms/step
Epoch 4/5
591/591 - 75s - loss: 1.3914 - accuracy: 0.5265 - val_loss: 2.9125 - val_accuracy: 0.2773 - 75s/epoch - 127ms/step
Epoch 5/5
591/591 - 75s - loss: 1.3289 - accuracy: 0.5563 - val_loss: 3.7754 - val_accuracy: 0.2551 - 75s/epoch - 127ms/step


In [ ]:
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5
591/591 - 75s - loss: 1.2766 - accuracy: 0.5691 - val_loss: 3.7532 - val_accuracy: 0.2727 - 75s/epoch - 127ms/step
Epoch 2/5
591/591 - 75s - loss: 1.2283 - accuracy: 0.5900 - val_loss: 3.7850 - val_accuracy: 0.2834 - 75s/epoch - 127ms/step
Epoch 3/5
591/591 - 75s - loss: 1.1870 - accuracy: 0.6074 - val_loss: 3.8169 - val_accuracy: 0.3008 - 75s/epoch - 127ms/step
Epoch 4/5
591/591 - 75s - loss: 1.1520 - accuracy: 0.6098 - val_loss: 3.8204 - val_accuracy: 0.2768 - 75s/epoch - 127ms/step
Epoch 5/5


With batch size of 16, about 17% GPU utilization (~1TFLOP) and 75 seconds/epoch

takes 7 epochs to perform better than random guessing (10% accuracy)

takes 9 epochs to achieve better than 50% accuracy